In [46]:
import pandas as pd
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [47]:
df = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/combined_books.csv")

/var/folders/dw/fn12lc3x2b317ydc5b70v2340000gn/T/ipykernel_85713/797220179.py:1: DtypeWarning: Columns (1,3,4,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/combined_books.csv")


## We will pre-process the data and see what are all the missing and duplicate values present in the dataset

In [48]:
df.isnull().sum()

user_id              97108
isbn                 38923
rating               24441
title                24442
author               24441
year                 97108
publisher            97108
age                 113480
normalized_title     26196
matched_genres       24441
desc                 58546
dtype: int64

### Columns that should NOT be missing (ideally):
rating - Drop rows where rating is missing

title -  Drop rows where title is missing

author - Drop rows where author is missing

matched_genres - Drop rows where matched_genres is missing

In [49]:
required_cols = ['rating', 'title', 'author', 'matched_genres']
df = df.dropna(subset=required_cols).copy()


In [50]:
df.shape

(127332, 11)

In [51]:
df['normalized_title'] = df['normalized_title'].fillna(df['title'].str.lower())


In [52]:
df.shape

(127332, 11)

In [53]:
df = df.drop(columns=['user_id', 'age', 'publisher', 'year'])


In [54]:
df.columns

Index(['isbn', 'rating', 'title', 'author', 'normalized_title',
       'matched_genres', 'desc'],
      dtype='object')

In [55]:
def inspect_all_dirty_strings(df):
    cols_to_check = [
        'isbn', 'rating', 'title', 'author',
        'normalized_title', 'matched_genres', 'desc'
    ]
    
    for col in cols_to_check:
        print(f"\n--- Inspecting column: {col} ---")
        
        # Convert to string and drop missing
        df_col = df[col].dropna().astype(str)
        
        print("Non-null count:", df_col.shape[0])
        print("Unique values:", df_col.nunique())

        # Check for only-numeric strings or very short strings
        print("\nSuspicious values (pure numbers or < 3 chars):")
        suspicious = df_col[df_col.str.match(r'^\d+$') | df_col.str.len().lt(3)]
        print(suspicious.value_counts().head(10))

        # Check for null-like values stored as strings
        print("\nString values that look like nulls ('nan', 'none', etc.):")
        null_like = df_col[df_col.str.lower().isin(['nan', 'none', 'null', 'n/a', 'na'])].value_counts()
        print(null_like)

        # Top 5 most common values
        print("\nTop 5 most frequent entries:")
        print(df_col.value_counts().head(5))
        
        print("-" * 60)


In [56]:
inspect_all_dirty_strings(df)



--- Inspecting column: isbn ---
Non-null count: 112851
Unique values: 88793

Suspicious values (pure numbers or < 3 chars):
isbn
0061015725    462
0316777730    268
0515122734    217
0440224675    202
1551667509    200
0446611808    200
0345378490    196
0345417623    195
0451184963    162
0440176484    154
Name: count, dtype: int64

String values that look like nulls ('nan', 'none', etc.):
Series([], Name: count, dtype: int64)

Top 5 most frequent entries:
isbn
0061015725    462
0316777730    268
0515122734    217
0440224675    202
1551667509    200
Name: count, dtype: int64
------------------------------------------------------------

--- Inspecting column: rating ---
Non-null count: 127332
Unique values: 290

Suspicious values (pure numbers or < 3 chars):
Series([], Name: count, dtype: int64)

String values that look like nulls ('nan', 'none', etc.):
Series([], Name: count, dtype: int64)

Top 5 most frequent entries:
rating
4.0    10317
3.5     6057
5.0     5576
4.5     5418
2.5   

### Cleaning Summary & Recommendations
isbn
Mostly clean. Looks like valid ISBNs (all numeric, fixed length).
Optional: Remove rows where isbn is missing if you plan to use it as a unique book key.

rating
We'll drop rows where rating is missing for collaborative filtering.

title / normalized_title
Some entries are very short (like "It", "V", "14"), which may be ambiguous or junk.
Keep short ones like “1984”, “It” (real books).
Remove/flag single-character or number-only titles like "1", "911", "Q".

author
A few numeric or junk entries ("19", "J.", "Ai") so replace them with empty author
Keep known short names if verified (Ai might be real).

matched_genres
Drop rows where matched_genres == '[]' (these can't be used in genre filtering).

for description
Remove junk entries like ".", ">", "No", "a", "PB" and make the value empty instead


In [57]:
# Drop rows missing required columns (except isbn)
df = df.dropna(subset=['rating', 'title', 'author', 'matched_genres']).copy()


In [58]:
# Convert columns to string for safe processing
df['title'] = df['title'].astype(str)
df['author'] = df['author'].astype(str)
df['matched_genres'] = df['matched_genres'].astype(str)
df['desc'] = df['desc'].astype(str)
df['normalized_title'] = df['normalized_title'].astype(str)


In [59]:
# Step 1: Remove titles that are purely numeric or single character (but keep legit short ones like "1984", "It")
legit_short_titles = {'it', 'we', 's.', 'v.', 'v', 'go'}
def is_bad_title(title):
    t = title.strip().lower()
    return ((len(t) <= 3 or t.isdigit()) and t not in legit_short_titles)


In [60]:
df = df[~df['title'].apply(is_bad_title)]


In [61]:
# Step 2: Replace junk author names (numbers or 1-2 character codes) with empty string
def clean_author(author):
    a = author.strip()
    if a.lower() in {'nan', 'none'} or a.isdigit() or len(a) <= 2 or a.lower() in {'j.', 'a.', 'b.'}:
        return ''
    return a
df['author'] = df['author'].apply(clean_author)


In [62]:
# Step 3: Drop rows where matched_genres is just an empty list
df = df[df['matched_genres'].str.strip() != '[]']


In [63]:
# Step 4: Replace junk descriptions with empty string
junk_descs = {'.', '>', 'no', 'b', 'a', 'pb', 'Â '}
df['desc'] = df['desc'].apply(lambda x: '' if x.strip().lower() in junk_descs or len(x.strip()) < 10 else x)


In [64]:
# Step 5: Normalize title if not available
df['normalized_title'] = df['normalized_title'].replace('nan', np.nan)
df['normalized_title'] = df['normalized_title'].fillna(df['title'].str.lower())


In [65]:
# Reset index for safety
df = df.reset_index(drop=True)


In [66]:
df.shape

(115758, 7)

In [ ]:
df['author'] = df['author'].apply(lambda x: x.title())

In [67]:
df.to_csv("/Users/whoseunassailable/Documents/coding_projects/college_projects/readiculous/data/processed/ready_for_feature_engineering.csv")